In [4]:
import os
import sys
import tensorflow as tf

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from ml.models.three_d import voxnet

DEBUG:root:Layer1:Conv3D shape=(None, 32, 14, 14, 14)
DEBUG:root:Layer3:Conv3D shape=(None, 32, 12, 12, 12)
DEBUG:root:Layer4:MaxPool3D shape=(None, 32, 6, 6, 6)
DEBUG:root:Layer5:Flatten shape=(None, 6912)
DEBUG:root:Layer6:Dense shape=(None, 128)
DEBUG:root:Layer8:Dense shape=(None, 2)


/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:79: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(input_shape=(1, 32, 32..., kernel_size=(5, 5, 5), filters=32, strides=(2, 2, 2), padding="valid", data_format="channels_first", kernel_initializer="normal", kernel_regularizer=<keras.reg..., bias_regularizer=<keras.reg...)`
  b_regularizer=l2(0.001),
/home/amy/elvo-analysis/venv/lib/python3.6/site-packages/keras/activations.py:186: UserWarning: Do not pass a layer instance (such as LeakyReLU) as the activation argument of another layer. Instead, advanced activation layers should be used just like any other layer in a model.
  identifier=identifier.__class__.__name__))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:87: UserWarning: Update your `Dropout` call to the Keras 2 API: `Dropout(rate=0.3)`
  self._mdl.add(Dropout(p=0.3))
/home/amy/elvo-analysis/ml/models/three_d/voxnet.py:99: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(kernel_size

INFO:root:Model compiled!


In [8]:
import os
import csv
import numpy as np
import random
from keras.preprocessing.image import ImageDataGenerator
from google.cloud import storage

BLACKLIST = []


class ChunkGenerator(object):
    def __init__(self, dims=(32, 32, 32), batch_size=16,
                 shuffle=True,
                 validation=False,
                 test=False, split_test=False,
                 split=0.2):
        self.dims = dims
        self.batch_size = batch_size
        self.validation = validation

        self.datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.1,
            height_shift_range=0.1,
            zoom_range=[1.0, 1.1],
        )

        # Access Google Cloud Storage
        gcs_client = storage.Client.from_service_account_json(
            '/home/amy/credentials/client_secret.json'
        )
        bucket = gcs_client.get_bucket('elvos')

        # Get label data from Google Cloud Storage
        blob = storage.Blob('annotated_labels.csv', bucket)
        blob.download_to_filename('tmp/augmented_annotated_labels.csv')
        prelim_label_data = {}
        with open('tmp/augmented_annotated_labels.csv', 'r') as pos_file:
            reader = csv.reader(pos_file, delimiter=',')
            for row in reader:
                if row[1] != 'label':
                    prelim_label_data[row[0]] = int(row[1])

        # Get 12096 random negatives from the label data to feed into our generator
        negative_counter = 0
        negative_label_data = {}
        print("getting 12096 random negative labels")
        while negative_counter < 12096:
            id_, label = random.choice(list(prelim_label_data.items()))
            if label == 0:
                negative_label_data[id_] = label
                del prelim_label_data[id_]
                negative_counter += 1
                
        # Get all of the positives from the label data
        label_data = {}
        print('getting positive labels')
        for id_, label in list(prelim_label_data.items()):
            if label == 1:
                for i in range(1, 25):
                    label_data[id_ + '_' + str(i)] = label
        
        # Put pos and neg together into one dictionary
        label_data.update(negative_label_data)

        # Get positive chunks
        print('getting positive chunks')
        pos_blobs = bucket.list_blobs(prefix='chunk_data/normal/positive')
        files = []
        for blob in pos_blobs:
            file = blob.name
            file = file.split('/')[-1]
            file = file.split('.')[0]

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            if not blacklisted:
                # Add all data augmentation methods
                files.append({
                    "name": file,
                })

        print("getting negative chunks")
        # Get negative chunks that were chosen to be in the labels
        neg_blobs = bucket.list_blobs(prefix='chunk_data/normal/negative')
        for blob in neg_blobs:
            file = blob.name

            # Check blacklist
            blacklisted = False
            for each in BLACKLIST:
                if each in file:
                    blacklisted = True

            file_id = blob.name.split('/')[-1]
            file_id = file_id.split('.')[0]

            if file_id in negative_label_data and not blacklisted:
                files.append({
                    "name": file,
                })

        # convert labels from dict to np array
        print('converting labels from dict to np array')
        labels = np.zeros(len(files))
        for i, file in enumerate(files):
            filename = file['name']
            filename = filename.split('/')[-1]
            filename = filename.split('.')[0]
            labels[i] = label_data[filename]

        # Take into account shuffling
        if shuffle:
            tmp = list(zip(files, labels))
            random.shuffle(tmp)
            files, labels = zip(*tmp)
            labels = np.array(labels)

        # Split based on validation
        print('splitting based on validation split')
        if validation:
            if split_test:
                files = files[:int(len(files) * split / 2)]
                labels = labels[:int(len(labels) * split / 2)]
            else:
                files = files[:int(len(files) * split)]
                labels = labels[:int(len(labels) * split)]
        elif test:
            if split_test:
                files = files[int(len(files) * split / 2):
                              int(len(files) * split)]
                labels = labels[int(len(labels) * split / 2):
                                int(len(labels) * split)]
            else:
                raise ValueError('must set split_test to True if test')
        else:
            files = files[int(len(files) * split):]
            labels = labels[int(len(labels) * split):]
        print(np.shape(files))
        print(np.shape(labels))
        print("Negatives: {}".format(np.count_nonzero(labels == 0)))
        print("Positives: {}".format(np.count_nonzero(labels)))
        self.files = files
        self.labels = labels
        self.bucket = bucket

    def generate(self):
        steps = self.get_steps_per_epoch()
        # print(steps)
        while True:
            for i in range(steps):
                # print(i)
                # print("D")
                x, y = self.__data_generation(i)
                yield x, y

    def get_steps_per_epoch(self):
        return len(self.files) // self.batch_size

    def __data_generation(self, i):
        bsz = self.batch_size
        files = self.files[i * bsz:(i + 1) * bsz]
        labels = self.labels[i * bsz:(i + 1) * bsz]
        images = []

        # Download files to tmp/npy/
        for i, file in enumerate(files):
            file_id = file['name'].split('/')[-1]
            file_id = file_id.split('.')[0]
            print(file_id)
            img = np.load('tmp/npy/{}.npy'.format(file_id))
            # print(np.shape(img))
            images.append(img)
        images = np.array(images)
        # print("Loaded entire batch.")
        # print(np.shape(images))
        return images, labels



In [9]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [10]:
train_gen = ChunkGenerator()
val_gen = ChunkGenerator(validation=True)

DEBUG:urllib3.util.retry:Converted retries value: 3 -> Retry(total=3, connect=None, read=None, redirect=None, status=None)
DEBUG:google.auth.transport.requests:Making request: POST https://accounts.google.com/o/oauth2/token
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): accounts.google.com:443
DEBUG:urllib3.connectionpool:https://accounts.google.com:443 "POST /o/oauth2/token HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.googleapis.com:443
DEBUG:urllib3.connectionpool:https://www.googleapis.com:443 "GET /storage/v1/b/elvos?projection=noAcl HTTP/1.1" 200 338


FileNotFoundError: [Errno 2] No such file or directory: 'tmp/augmented_annotated_labels.csv'

In [ ]:
# Train model on dataset
model.fit_generator(generator=train_gen.generate(),
                    validation_data=val_gen.generate(),
                    steps_per_epoch=training_generator.get_steps_per_epoch(),
                    validation_steps=validation_generator.get_steps_per_epoch(),
                    epochs = 15)